In [34]:
import os, glob
import pandas as pd
from io import StringIO
import numpy as np
import pysam
import pybedtools 
from Bio import SeqIO
from Bio.Seq import Seq
pybedtools.helpers.set_tempdir("/data/projects/temp")

In [35]:
non_coding_region = "acceptor"

In [36]:
data_path = '/data/projects/DNABERT_snv/Manuscript_11_2023/Intersected_data/{}_ICGC_intersected.tsv'.format(non_coding_region)
output_path = "/data/projects/DNABERT_snv/Manuscript_11_2023/DNABERT_data/{}".format(non_coding_region) 

In [37]:
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [38]:
def seq2kmer(seq):
    """
    Convert original sequence to kmers
    
    Arguments:
    seq -- str, original sequence.
    k -- int, kmer of length k specified.
    
    Returns:
    kmers -- str, kmers separated by space
    """
    k=6
    kmer = [seq[x:x+k] for x in range(len(seq)+1-k)]
    kmers = " ".join(kmer)
    return kmers

In [39]:
genome = SeqIO.to_dict(SeqIO.parse("/data/projects/Resources/Gencode_genome_annotation/GRCh38.primary_assembly.genome.fa", "fasta"))

In [40]:
df = pd.read_csv(data_path, sep= '\t')
print(df.shape)

(6001604, 22)


In [41]:
# df = pd.read_csv(data_path, sep= '\t')
# print(df.shape)

# df = df[df['No_of_variations']<3].reset_index(drop=True)
# print(df.shape)
# df = df.loc[df['Alternative_Allele']== '.'].reset_index(drop=True)
# df
#df= df[df['strand']=="-"]


data= []
df = df.head(10)
# Iterate over DataFrame rows
for idx, row in df.iterrows():
    try:
        #print(row)Reference_Allele	Alternative_Allele
        alts = row['Alternative_Allele'].rstrip(',').split(',')
        ref_seq = str(genome[row['chr_name']].seq[ row['Acceptor_start']:row['Acceptor_end']])

        # Calculate the variant position relative to the fetched sequence
        variant_pos_start = row['Start'] - row['Acceptor_start']
        variant_pos_end = row['End']  - row['Acceptor_start']
        #print(variant_pos_start, variant_pos_end, alts,  row['Reference_Allele'])
        # print(ref_seq)
        # if row['strand'] == '-':
        #     ref_seq = str(Seq(ref_seq).reverse_complement())
        #print(ref_seq)


        #Replace the reference nucleotide with the alternate nucleotide to get the alternate sequence
        for alt in alts:
            if alt == ".":  # Deletion
                # Fetch additional bases from genome to maintain sequence length
                extra_bases = str(genome[row['chr_name']].seq[row['Acceptor_end']:row['Acceptor_end'] + len(row['Reference_Allele'])])
                #print(len(row['Reference_Nucleotide']))
                alt_seq = ref_seq[:variant_pos_start] + ref_seq[variant_pos_end:] + extra_bases
            else:  # SNPs, Insertions, and partial Deletions
                extra_bases = str(genome[row['chr_name']].seq[row['Acceptor_end']:row['Acceptor_end'] + len(row['Reference_Allele']) - len(alt)])
                #print(extra_bases)
                if(variant_pos_start>=0):
                    alt_seq = ref_seq[:variant_pos_start] + alt + ref_seq[variant_pos_end:] + extra_bases
                else:
                    continue
            #print(alt_seq)
            #Append to the list as a dictionary
            data.append({
                'chr': row['chr_name'],
                'strand': row['strand'],
                'acceptor_coordinates': str(row['Acceptor_start'])+'-'+str(row['Acceptor_end']),
                'Ensemble_Transcript_ID': row['Transcript_ID'],
                'Mutation_ID': row['Mutation_ID'],
                'varinat_start':  row['Start'],
                'variant_end':  row['End'],
                'ref_neucleotide': row['Reference_Allele'],
                'alternative_neucleotide': alt,
                'reference_seq': ref_seq,
                'alt_seq': alt_seq
            })
            
    except KeyError:
        print("NA")
        # # If sequence is not present, append a default value
        # ref_sequences.append("NA")
        # alt_sequences.append("NA")
        
new_df = pd.DataFrame(data)
print(new_df.shape)
new_df = new_df.drop_duplicates().reset_index(drop=True)
print(new_df.shape)
merged_list = list(zip(new_df['reference_seq'], new_df['alt_seq']))
merged_list = [item.upper() for tup in merged_list for item in tup]
#print(merged_list)
kmer_lst = list(map(seq2kmer, merged_list))
df_kmer = pd.DataFrame(kmer_lst, columns=['Sequence'])
df_kmer['Label'] = np.random.choice([0, 1], size=len(df_kmer))

new_df.to_csv(output_path + "/all_data_new.tsv", sep="\t", index= False)
df_kmer.to_csv(output_path + "/dev.tsv", sep="\t", index= False)

(5995702, 11)
(5992039, 11)


In [42]:
new_df

,chr,strand,acceptor_coordinates,Ensemble_Transcript_ID,Mutation_ID,varinat_start,variant_end,ref_neucleotide,alternative_neucleotide,reference_seq,alt_seq
0,chr7,+,127589043-127589122,ENST00000000233,MU82778890,127589078,127589079,G,T,TCGCTCCCATCTCCATCCCTGTGCCCCTTTCCGTTGCAGTTGGCTT...,TCGCTCCCATCTCCATCCCTGTGCCCCTTTCCGTTTCAGTTGGCTT...
1,chr7,+,127589043-127589122,ENST00000000233,MU85932120,127589090,127589091,G,T,TCGCTCCCATCTCCATCCCTGTGCCCCTTTCCGTTGCAGTTGGCTT...,TCGCTCCCATCTCCATCCCTGTGCCCCTTTCCGTTGCAGTTGGCTT...
2,chr7,+,127589043-127589122,ENST00000000233,MU84868715,127589096,127589097,G,T,TCGCTCCCATCTCCATCCCTGTGCCCCTTTCCGTTGCAGTTGGCTT...,TCGCTCCCATCTCCATCCCTGTGCCCCTTTCCGTTGCAGTTGGCTT...
3,chr7,+,127589043-127589122,ENST00000000233,MU82319183,127589104,127589105,G,T,TCGCTCCCATCTCCATCCCTGTGCCCCTTTCCGTTGCAGTTGGCTT...,TCGCTCCCATCTCCATCCCTGTGCCCCTTTCCGTTGCAGTTGGCTT...
4,chr7,+,127588371-127588450,ENST00000000233,MU133310861,127588386,127588387,C,A,GAGGGCAGCGACGCGCGGAGGCGGCGGCGGAGCCTCCTCCTGCTGC...,GAGGGCAGCGACGCGAGGAGGCGGCGGCGGAGCCTCCTCCTGCTGC...
...,...,...,...,...,...,...,...,...,...,...,...
5992034,chr17,-,63713488-63713567,ENST00000640999,MU84976091,63713494,63713495,G,T,TTGCTAGATTCACAGTCATCAGGTCCTCAAATCCTTTGCCTAAAAG...,TTGCTATATTCACAGTCATCAGGTCCTCAAATCCTTTGCCTAAAAG...
5992035,chr17,-,63713488-63713567,ENST00000640999,MU129479225,63713502,63713503,G,A,TTGCTAGATTCACAGTCATCAGGTCCTCAAATCCTTTGCCTAAAAG...,TTGCTAGATTCACAATCATCAGGTCCTCAAATCCTTTGCCTAAAAG...
5992036,chr17,-,63713488-63713567,ENST00000640999,MU29299852,63713515,63713516,C,G,TTGCTAGATTCACAGTCATCAGGTCCTCAAATCCTTTGCCTAAAAG...,TTGCTAGATTCACAGTCATCAGGTCCTGAAATCCTTTGCCTAAAAG...
5992037,chr17,-,63713488-63713567,ENST00000640999,MU121174896,63713550,63713551,A,T,TTGCTAGATTCACAGTCATCAGGTCCTCAAATCCTTTGCCTAAAAG...,TTGCTAGATTCACAGTCATCAGGTCCTCAAATCCTTTGCCTAAAAG...


In [32]:
df_kmer

,Sequence,Label
0,TCGCTC CGCTCC GCTCCC CTCCCA TCCCAT CCCATC CCAT...,0
1,TCGCTC CGCTCC GCTCCC CTCCCA TCCCAT CCCATC CCAT...,0
2,TCGCTC CGCTCC GCTCCC CTCCCA TCCCAT CCCATC CCAT...,1
3,TCGCTC CGCTCC GCTCCC CTCCCA TCCCAT CCCATC CCAT...,0
4,TCGCTC CGCTCC GCTCCC CTCCCA TCCCAT CCCATC CCAT...,1
5,TCGCTC CGCTCC GCTCCC CTCCCA TCCCAT CCCATC CCAT...,0
6,TCGCTC CGCTCC GCTCCC CTCCCA TCCCAT CCCATC CCAT...,0
7,TCGCTC CGCTCC GCTCCC CTCCCA TCCCAT CCCATC CCAT...,1
8,GAGGGC AGGGCA GGGCAG GGCAGC GCAGCG CAGCGA AGCG...,1
9,GAGGGC AGGGCA GGGCAG GGCAGC GCAGCG CAGCGA AGCG...,0
